In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

## Data

In [2]:
data = pd.read_csv('/Users/genie/data/covid-19/owid-covid-data.csv')
data = data[(data.iso_code.notnull()) & ~(data.iso_code.isin(['OWID_KOS','OWID_WRL']))]
data = data.rename(columns={'location':'country_name'})

## Data Viz - Daily New cases

In [3]:
df = data

df = df.sort_values(by=['date'])
df['new_cases'] = df.apply(lambda x:x['new_cases'] if x['new_cases'] >=0 else np.nan, axis=1)

In [4]:
fig = px.choropleth(df, locations="iso_code",
                    color="new_cases",
                    hover_name="country_name",
                    animation_frame="date",
                    title = "Daily new COVID cases",
                    color_continuous_scale=px.colors.sequential.OrRd)

fig["layout"].pop("updatemenus")
# fig.show()
fig.write_html("daily_new_cases.html")

## Data Viz - Time taken for the first case and first 1000 cases

In [5]:
df = data[(data.iso_code.notnull()) & ~(data.iso_code.isin(['OWID_KOS','OWID_WRL']))].copy()

first_case_data = df[df.total_cases > 0].groupby(['iso_code'])['date'].min().reset_index()
first_case_data = first_case_data.rename(columns={'date':'first_case_date'})
df = pd.merge(df, first_case_data, how='left', on=['iso_code'])

df['days_until_first_case'] = (pd.to_datetime(df['first_case_date'])-pd.to_datetime(df['date'].min())).dt.days

cases_1000_data = df[df.total_cases>=1000].groupby(['iso_code'])['date'].min().reset_index()
cases_1000_data = cases_1000_data.rename(columns={'date':'cases_1000_date'})
df = pd.merge(df, cases_1000_data, how='left', on=['iso_code'])
df['days_until_1000_cases'] = (pd.to_datetime(df['cases_1000_date'])-pd.to_datetime(df['first_case_date'])).dt.days

df = df[['iso_code','country_name','first_case_date','days_until_first_case','cases_1000_date','days_until_1000_cases']].drop_duplicates()

del first_case_data
del cases_1000_data

In [6]:
fig = px.choropleth(df, locations="iso_code",
                    color="days_until_first_case",
                    hover_name="country_name",
                    title = "Days until the first reported case",
                    color_continuous_scale='OrRd_r')

fig.write_html("days_until_first_case.html")

In [7]:
fig = px.choropleth(df, locations="iso_code",
                    color="days_until_1000_cases",
                    hover_name="country_name",
                    title = "Days until 1000 cases",
                    color_continuous_scale='OrRd_r')

fig.write_html("days_until_1000_cases.html")